In [1]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-11 15:29:05.790283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749644945.880627    3462 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749644945.906791    3462 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749644946.102814    3462 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749644946.102831    3462 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749644946.102834    3462 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 4120

# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-3B-Instruct",
    max_seq_length=max_seq_length,
    load_in_8bit=False,
    load_in_4bit=False,
    full_finetuning=False,
    device_map="auto"
)

==((====))==  Unsloth 2025.4.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla V100-SXM2-32GB. Num GPUs = 1. Max memory: 31.733 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=False,
    # use_gradient_checkpointing="unsloth",
    random_state=42,
    loftq_config=None,
)

print(model.print_trainable_parameters())

Unsloth 2025.4.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


trainable params: 29,933,568 || all params: 3,115,872,256 || trainable%: 0.9607
None


In [4]:
import json

def read_jsonl(file_name):
    with open(file_name, encoding="utf-8") as r:
        return [json.loads(line) for line in r]
    
data = read_jsonl("../data/train/webglm_raft_train.jsonl")
val_data = read_jsonl("../data/val/webglm_raft_val.jsonl")

In [5]:
for i in data:
    print(i)
    break

{'messages': [{'content': 'Ты — экспертная система Compressa RAG, предоставляющая точные и релевантные ответы на вопросы, используя только предоставленную контекстную информацию. Отвечай на русском языке.', 'role': 'system'}, {'content': '# Контекстная информация:\n\nИсточник [1]:\n"Почему важно размышлять об этой истине? медитировать на этом освобождает, потому что это подразумевает, что то, что мы лично живем внутри наших голов, не является реальным. мы осознаем наши мысли, так что в этом смысле они реальны. и все же наши мысли не существуют вне нашего сознания. нет нигде, где мысль, которая появляется в вас в этот момент, на самом деле происходит. в отличие от того, как мы представляем себе это, наши мысли не являются твердыми, как деревья или камни, которые существуют вне нас каким-то ощутимым образом. конечно, я никогда не видел мысли, ходя по улице."""\n\nИсточник [2]:\nВ случае гравитации, пространства и двух объектов все три действуют вместе, чтобы создать это притяжение.\n\nИс

In [6]:
from datasets import Dataset
dataset = Dataset.from_list(data)
val_dataset = Dataset.from_list(val_data)




train = tokenizer.apply_chat_template(
    dataset["messages"],
    tokenize=False,
)

val = tokenizer.apply_chat_template(
    val_dataset["messages"],
    tokenize=False,
)

In [7]:
# custom_system_message = {
#     "role": "system", 
#     "content": "Ты — экспертная система Compressa RAG. Предоставляющая точные и релевантные ответы на вопросы."
# }

# # def generate_conversation(examples):
# #     conversations = []
# #     for messages in examples["messages"]:
# #         formatted_messages = [
# #             {"role": msg["role"], "content": msg["content"]}
# #             for msg in messages
# #         ]
# #         conversations.append(formatted_messages)
# #     return {"conversations": conversations}

# def generate_conversation(examples):
#     conversations = []
#     for messages in examples["messages"]:
#         formatted_messages = [custom_system_message] + [  # <- кастомный промпт
#             {"role": msg["role"], "content": msg["content"]}
#             for msg in messages
#         ]
#         conversations.append(formatted_messages)
#     return {"conversations": conversations}

# train = tokenizer.apply_chat_template(
#     dataset.map(generate_conversation, batched=True)["conversations"],
#     tokenize=False,
# )

# val = tokenizer.apply_chat_template(
#     val_dataset.map(generate_conversation, batched=True)["conversations"],
#     tokenize=False,
# )


In [8]:
print(train[0])

<|im_start|>system
Ты — экспертная система Compressa RAG, предоставляющая точные и релевантные ответы на вопросы, используя только предоставленную контекстную информацию. Отвечай на русском языке.<|im_end|>
<|im_start|>user
# Контекстная информация:

Источник [1]:
"Почему важно размышлять об этой истине? медитировать на этом освобождает, потому что это подразумевает, что то, что мы лично живем внутри наших голов, не является реальным. мы осознаем наши мысли, так что в этом смысле они реальны. и все же наши мысли не существуют вне нашего сознания. нет нигде, где мысль, которая появляется в вас в этот момент, на самом деле происходит. в отличие от того, как мы представляем себе это, наши мысли не являются твердыми, как деревья или камни, которые существуют вне нас каким-то ощутимым образом. конечно, я никогда не видел мысли, ходя по улице."""

Источник [2]:
В случае гравитации, пространства и двух объектов все три действуют вместе, чтобы создать это притяжение.

Источник [3]:
Требуется о

In [9]:
# tokens_count = 0
# arr = []

# for idx, line in enumerate(train):
#     l = len(tokenizer.encode(line, add_special_tokens=False))
#     tokens_count += l
#     arr.append(l)

In [10]:
# import numpy as np

# arr = np.sort(np.array(arr))

In [11]:
# arr

In [12]:
# print(f"tokens_avg {tokens_count / len(train)}")
# print(f"tokens_max {arr[-1]}")
# print(f"tokens_q95 {arr[int(len(train)*0.95)]}")
# print(f"tokens_q99 {arr[int(len(train)*0.99)]}")

In [6]:
len(train), len(val)

(47368, 1100)

In [7]:
import pandas as pd
from datasets import Dataset


train_tmp = pd.Series(train)
val_tmp = pd.Series(val)

train_tmp.name = "text"
val_tmp.name = "text"

train_dataset = Dataset.from_pandas(pd.DataFrame(train_tmp))
train_dataset = train_dataset.shuffle(seed = 228)
val_dataset = Dataset.from_pandas(pd.DataFrame(val_tmp))

In [8]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    packing = False,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 3,
        gradient_accumulation_steps = 8,
        per_device_eval_batch_size = 4,
        warmup_steps = 25,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 228,
        report_to="wandb",
        output_dir="qwen3b-v100-raft",
        eval_steps=1000,
        eval_strategy="steps",
        dataloader_num_workers=10,
        save_steps=75
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=20):   0%|          | 0/47368 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=20):   0%|          | 0/1100 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 47,368 | Num Epochs = 1 | Total steps = 1,973
O^O/ \_/ \    Batch size per device = 3 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (3 x 8 x 1) = 24
 "-____-"     Trainable parameters = 29,933,568/3,115,872,256 (0.96% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: seba-vicin (seba-vicin-oxford) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [9]:
torch.save(model.state_dict(), "qwen3b-v100-webglm-raft/model.pth")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f2278b190c0>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f2278b190c0>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [10]:
model.save_pretrained("qwen3b-v100-webglm-raft/pretrain_save")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f2278b190c0>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f2278b190c0>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [10]:
torch.cuda.empty_cache()

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7fa2c2738790>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7fa2c2738790>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe